In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 50% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_50, _, y_50 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.5, stratify=y_label_0_1, random_state=42)

In [6]:
X_50.shape

(455884, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt50 = DecisionTreeClassifier()
dt50.fit(X_50, y_50)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  3211    155   2280    787     67   1083    186 105928    189     85]
(3211, 56) (3211,)
Save 0 to baseline_Def1.npy
(155, 56) (155,)
Save 1 to baseline_Def2.npy
(2280, 56) (2280,)
Save 2 to baseline_Def3.npy
(787, 56) (787,)
Save 3 to baseline_Def4.npy
(67, 56) (67,)
Save 4 to baseline_Def5.npy
(1083, 56) (1083,)
Save 5 to baseline_Def6.npy
(186, 56) (186,)
Save 7 to baseline_Def8.npy
(105928, 56) (105928,)
Save 8 to baseline_Def9.npy
(189, 56) (189,)
Save 9 to baseline_Def10.npy
(85, 56) (85,)
Save 10 to baseline_Def11.npy
Execution Time: 0.390833 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [108037     67   1156   1549    881 165412    107  64680     13     11]
(108037, 56) (108037,)
Save 0 to BIM_Def1.npy
(67, 56) (67,)
Save 1 to BIM_Def2.npy
(1156, 56) (1156,)
Save 2 to BIM_Def3.npy
(1549, 56) (1549,)
Save 3 to BIM_Def4.npy
(881, 56) (881,)
Save 4 to BIM_Def5.npy
(165412, 56) (165412,)
Save 5 to BIM_Def6.npy
(107, 56) (107,)
Save 7 

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  4118    215   2311   1064    119   1536    261 104107    150     90]
(4118, 56) (4118,)
Save 0 to baseline_Def1.npy
(215, 56) (215,)
Save 1 to baseline_Def2.npy
(2311, 56) (2311,)
Save 2 to baseline_Def3.npy
(1064, 56) (1064,)
Save 3 to baseline_Def4.npy
(119, 56) (119,)
Save 4 to baseline_Def5.npy
(1536, 56) (1536,)
Save 5 to baseline_Def6.npy
(261, 56) (261,)
Save 7 to baseline_Def8.npy
(104107, 56) (104107,)
Save 8 to baseline_Def9.npy
(150, 56) (150,)
Save 9 to baseline_Def10.npy
(90, 56) (90,)
Save 10 to baseline_Def11.npy
Execution Time: 1.335287 seconds
(341913,)
[0 2 3 4 5 7 8 9] [131820    516    790      5 176279     10  32488      5]
(131820, 56) (131820,)
Save 0 to BIM_Def1.npy
(516, 56) (516,)
Save 2 to BIM_Def3.npy
(790, 56) (790,)
Save 3 to BIM_Def4.npy
(5, 56) (5,)
Save 4 to BIM_Def5.npy
(176279, 56) (176279,)
Save 5 to BIM_Def6.npy
(10, 56) (10,)
Save 7 to BIM_Def8.npy
(32488, 56) (32488,)
Save 8 to BIM_Def9.npy
(5, 56) (5,)

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  5680    365   2474   1055    180   1864    409 101629    163    152]
(5680, 56) (5680,)
Save 0 to baseline_Def1.npy
(365, 56) (365,)
Save 1 to baseline_Def2.npy
(2474, 56) (2474,)
Save 2 to baseline_Def3.npy
(1055, 56) (1055,)
Save 3 to baseline_Def4.npy
(180, 56) (180,)
Save 4 to baseline_Def5.npy
(1864, 56) (1864,)
Save 5 to baseline_Def6.npy
(409, 56) (409,)
Save 7 to baseline_Def8.npy
(101629, 56) (101629,)
Save 8 to baseline_Def9.npy
(163, 56) (163,)
Save 9 to baseline_Def10.npy
(152, 56) (152,)
Save 10 to baseline_Def11.npy
Execution Time: 0.097712 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [171467    570   7020   5527   2382 118082     15   6740  29908     77
    125]
(171467, 56) (171467,)
Save 0 to BIM_Def1.npy
(570, 56) (570,)
Save 1 to BIM_Def2.npy
(7020, 56) (7020,)
Save 2 to BIM_Def3.npy
(5527, 56) (5527,)
Save 3 to BIM_Def4.npy
(2382, 56) (2382,)
Save 4 to BIM_Def5.npy
(118082, 56) (118082,)
Save 5 to BIM_Def6.npy
(15

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ExcludeCaFA/UNSW_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_ExcludeCaFA.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,3211,155,2280,787,67,1083,0,186,105928,189,85,113971
1,XGB50,108037,67,1156,1549,881,165412,0,107,64680,13,11,341913
2,XGB50,102844,45,1160,1518,288,171708,0,829,63489,8,24,341913
3,XGB50,108037,67,1156,1549,881,165412,0,107,64680,13,11,341913
4,XGB50,74460,644,2250,311,180,243417,485,18053,1671,304,138,341913
5,XGB50,121948,341,4401,2465,943,165189,0,448,45481,453,244,341913
6,XGB50,6997,439,6757,2425,199,3034,0,558,320727,520,257,341913
7,XGB50,100428,22,824,1664,26,539,0,84,238290,12,24,341913
8,XGB50,114836,373,336,985,668,157308,0,70,67325,5,7,341913
9,XGB50,101643,49,1234,1353,791,170186,0,57,66568,10,22,341913
